<a href="https://colab.research.google.com/github/Kan4me/HackathonA3Data2021/blob/main/A3hackathon2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hackathon A3 Data 2021

## Objetivo

Desenvolver um pipeline de extração, transformação e disponibilização de dados. Além disso, responder de forma visual as seguintes questões:


1. Nos últimos 10 anos, quais foram os salários médios de homens e mulheres que
trabalham com tecnologia na região sudeste do Brasil por ano?
2. Nos últimos 10 anos, quais foram os salários médios das pessoas por nível de
escolaridade que trabalhavam no setor de agronegócio na região sul do Brasil?
3. Entre os setores de tecnologia, indústria automobilística e profissionais da
saúde, qual deles teve o maior crescimento? Qual foi o número de trabalhadores
em cada setor por ano?
4. Nos últimos 10 anos, quais foram os setores que possuem, em cada ano, o maior
número de trabalhadores que possuem jornada semanal inferior a 40h?
5. Qual é o número absoluto de pessoas por cada categoria de sexo que realizaram
trabalho intermitente em cada um dos últimos anos?

## Fonte de Dados

Microdados da RAIS (Ministério do Trabalho) dos últimos 10 anos - de 2010 a 2019

# Acesso aos dados


O acesso aos dados foi realizado de forma manual seguindo os seguintes passos:

1. Adiconando um Acesso de Rede de forma manual conforme o link ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/ e login anônimo;
2. Dados upados de forma manual no Google Drive pessoal.

# Conexão com o Google Drive e instalação dos pacotes utilizados

In [1]:
# pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
#Mounted Google Drive de interesse
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
# Configuração das bibliotecas de interesse

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os
import pyarrow as pa
import pyarrow.parquet as pq
!pip install py7zr

## Definiçao diretório de interesse no Google Drive

In [4]:
import py7zr

os.chdir("/content/drive/MyDrive/data.source/A3Data")

# Carregar dicionários auxiliares

Fontes dos dicionários listados abaixo:

* sex_label, school_label e age_label: planilha 'RAIS_vinculos_layout' disponibilizada no seguinte domínio: ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/



In [5]:
sex_label = {1: "M", 2: "F"}

school_label = {1:	"Analfabeto", 2:	"Até 5.a incomp", 3:	"Até 5.a compl", 
                4:	"6 a 9.a EF", 5:	"EF compl", 6: "EM incomp",
                7:	"EM compl", 8:	"ES incomp", 9: "ES compl", 10:	"Mestrado",
                11:	"Doutorado", -1:	"Ignorado"}

age_label = {1:	"10-14", 2:	"15-17", 3:	"18-24", 4:	"25-29", 5:	"30-39", 
             6:	"40-49", 7:	'50-64', 8:	"65+"}

# Tratamento dos dados - Por Ano

Colunas de interesse de 2010 a 2014:
* 'CNAE 2.0 Classe'
* 'Faixa Etária'
* 'Escolaridade após 2005'
* 'Qtd Hora Contr' - Quando tal valor é igual a 1, indica que o indivíduo realizou Trabalho Intermitente naquele ano. Fonte: [Manual da RAIS](http://www.rais.gov.br/sitio/rais_ftp/ManualRAIS2020.pdf)
* 'Vl Remun Média Nom'
* 'Sexo Trabalhador'


In [6]:
col_2010to2014 = ["CNAE 2.0 Classe", "Faixa Etária", "Escolaridade após 2005", 
            "Qtd Hora Contr", "Vl Remun Média Nom", "Sexo Trabalhador"]

## 2010


### Região Norte


In [ ]:
norte_2010_zip = ["AC2010.7z", "AP2010.7z", "AM2010.7z", "PA2010.7z", "RO2010.7z", 
                  "RR2010.7z", "TO2010.7z"]

for i in range(len(norte_2010_zip)):
  with py7zr.SevenZipFile(norte_2010_zip[i], mode='r') as z:
    z.extractall()

In [7]:
norte_2010_txt = ["AC2010.txt", "AP2010.txt", "AM2010.txt", "PA2010.txt", 
                  "RO2010.txt", "RR2010.txt", "TO2010.txt"]

df_norte_2010 = pd.DataFrame(columns= col_2010to2014)

for i in range(len(norte_2010_txt)):
  df = pd.read_csv(norte_2010_txt[i], encoding='ISO-8859-1', sep= ";", header= 0)
  df = df.loc[:, col_2010to2014]
  df_norte_2010 = df_norte_2010.append(df)

In [8]:
df_norte_2010["Ano"] = 2010
df_norte_2010["Sexo Trabalhador"] = df_norte_2010["Sexo Trabalhador"].replace(sex_label)
df_norte_2010["Escolaridade após 2005"] = df_norte_2010["Escolaridade após 2005"].replace(school_label)
df_norte_2010["Faixa Etária"] = df_norte_2010["Faixa Etária"].replace(age_label)
df_norte_2010["Vl Remun Média Nom"] = df_norte_2010["Vl Remun Média Nom"].str.replace(",", ".")
df_norte_2010["Vl Remun Média Nom"] = pd.to_numeric(df_norte_2010["Vl Remun Média Nom"])

df_norte_2010.head(15)

,CNAE 2.0 Classe,Faixa Etária,Escolaridade após 2005,Qtd Hora Contr,Vl Remun Média Nom,Sexo Trabalhador,Ano
0,16102,30-39,Analfabeto,44,765.00,M,2010
1,16102,30-39,Analfabeto,44,721.99,M,2010
2,16102,18-24,Analfabeto,44,595.50,M,2010
3,41204,30-39,Analfabeto,44,503.46,M,2010
4,16102,30-39,Analfabeto,44,792.93,M,2010
5,56201,18-24,Analfabeto,44,600.00,M,2010
6,41204,50-64,Analfabeto,44,718.65,M,2010
7,41204,50-64,Analfabeto,44,746.80,M,2010
8,41204,50-64,Analfabeto,44,659.02,M,2010
9,81214,25-29,Analfabeto,44,658.93,M,2010


**OBS:** Criar uma coluna no DataFrame final da mesorregião que informa o setor de atuação do indivíduo conforme o CNAE 2.0 apresentado. Ideia: usar regex para criar tal coluna.

In [9]:
df_norte_2010.dtypes

CNAE 2.0 Classe            object
Faixa Etária               object
Escolaridade após 2005     object
Qtd Hora Contr             object
Vl Remun Média Nom        float64
Sexo Trabalhador           object
Ano                         int64
dtype: object

**OBS:** Existem chances reais das varáveis acima apresentarem problemas nos demais passos do problema.

In [10]:
df_norte_2010.to_csv (r'df_norte_2010.csv', index = False, header=True)

Solução esperta para lidar com o problema de Volume dos dados:

* Utilizar outros frameworks para paralelizar e particionar as tarefas, tipo [PySpark](https://spark.apache.org/docs/latest/api/python/) e [Dask DataFrame](https://docs.dask.org/en/latest/);
* Salvar em parquet.

#### A partir desse ponto, é replicar a mesma lógica apresentada acima nas demais regiões do mesmo ano e nos demais anos, com bastante atenção nas colunas que devem ser utilizadas que possuem a mesma informação de interesse.

### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

## 2011

### Região Norte


### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

## 2012

### Região Norte


### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

## 2013

### Região Norte


### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

## 2014

### Região Norte


### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

## 2015

### Região Norte


### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

## 2016

### Região Norte


### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

## 2017

### Região Norte


### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

## 2018


### Região Norte


### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

## 2019

### Região Norte


### Região Nordeste

### Região Centro-Oeste

### Região Sudeste

### Região Sul

# Esboço Pipeline

Desenvolver os códigos com [Pdpipe](https://pdpipe.github.io/pdpipe/doc/pdpipe/) que façam as seguintes tarefas:
1. O tratamento de dados apresentado acima e demais refinamentos necessários;
2. Apresentação visual das questões solicitadas utilizando o [Plotly](https://plotly.com/python/).

# Principais problemas enfrentados

## Volume de dados: 


O serviço de armazenamento utilizado para desenvolver o presente desafio é Google Drive limitado a 15GB. 

Para desenvolver o desafio é necessário upar em uma pasta no armazenamento utilizado os arquivos de interesse, extrair os mesmos e depois salvar o data frame gerado no formato de interesse.

## Tratamento dos dados:

Para gerar os data frames de interesse, é necessário realizar diversas manipulações para manter e criar as informações relevantes do problema de interesse.

Uma manipulação solicitada no problema é definir os setores da indústria que o indivíduo pertence. Tal definição depende de pesquisas no site do [Cadastro Nacional de Atividade Econômica - CNAE](https://concla.ibge.gov.br/busca-online-cnae.html).

## Curva de Aprendizado

Para tratar os dados e desenvolver o pipeline do problema, é necessário um intervalo de tempo maior do que 5 (cinco) dias corridos. 